## Qiskit for k-coloring, NK method

from paper: A general quantum method to solve the graph K-colouring problem https://hal.science/hal-02891847v1

In [2]:
from qiskit import *
from qiskit_aer import QasmSimulator, AerError
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
import numpy as np
%matplotlib inline

In [3]:
# k-coloring of a graph
# N = number of nodes
# K = number of colours

nNodes = 3
nColors = 3  # nColors <= nNodes
nc = nColors + 1 
nn2 = round((nNodes-1)*nNodes/2)
sc = round(nc*nNodes)
sg = round(nc*nNodes + nn2)

nqubits = sc + 2*nn2 # Total number of qubits

# Creatr a Quantum Circuit
q = QuantumRegister(nqubits)
c = ClassicalRegister(nColors * nNodes)
qc = QuantumCircuit(q,c)
simulator = QasmSimulator()

# qc.x(sg) # Set to |1> iif there is an edge
# qc.x(sg+1)
# qc.x(sg+2)
# qc.x(sg+3)
# qc.x(sg+4)
# qc.x(sg+5)

# Initialisation
for n in range(nNodes):
    qc.x(sg+n)

s = 0
for n in range(nNodes):
    for k in range(nColors):
        qc.h(s+k)
    s = s + nc

# Constraints
s = 0
for n in range(nNodes):
    for k in range(nColors-1):
        for l in range(k+1, nColors):
            # Eliminate 11
            qc.ccx(s+k, s+l, s+nColors)
            qc.cx(s+nColors, s+k)
            qc.reset(s+nColors)

        # Eliminate 0* (no color assigned to the node n)
    for k in range(nColors):
        qc.x(s+k)
    
    cb = list(range(s, s+nColors))
    qc.mcx(cb,s+nColors)

    for k in range(nColors):
        qc.x(s+k)
        
    qc.cx(s+nColors, s+nColors-1)
    qc.reset(s+nColors)
    s = s + nc

print('end of colouring matrices')

for k in range(nColors):
    s = nc*nNodes
    for n1 in range(nNodes-1):
        for n2 in range(n1+1,nNodes):
            n11 = nc*n1+k
            n22 = nc*n2+k
            qc.ccx(n11,n22,s)
            s = s+1

print('end of pairs of nodes')


end of colouring matrices
end of pairs of nodes


In [4]:
# Compare to the graph
for n in range(sc,sc+nn2):
    for node in range(nNodes):
        qnode = nc*node
        qnc = qnode+nColors
        for k in range(nColors):
            cb = [n, n+nn2, qnode+k]
            qc.mcx(cb,qnc)
            cb = [n, n+nn2, qnc]
            qc.mcx(cb, qnode+k)
            qc.reset(qnc)
print('end of compare to graph')

end of compare to graph


In [5]:
# Measure
cb = 0
for n in range(nNodes):
    s = n*(nColors+1)
    for k in range(nColors):
        qb = s+k
        qc.measure(qb,cb)
        cb = cb +1
print('end of measure')
print(qc)
d = qc.depth()
print("Circuit depth: ", d)
print("Circuit width: ", nqubits)
print("Complexity: ", d*nqubits)

end of measure
       ┌───┐     ┌───┐          ┌───┐┌───┐                    ┌───┐          »
 q1_0: ┤ H ├──■──┤ X ├───────■──┤ X ├┤ X ├─────────────────■──┤ X ├───────■──»
       ├───┤  │  └─┬─┘       │  └─┬─┘└───┘     ┌───┐┌───┐  │  ├───┤       │  »
 q1_1: ┤ H ├──■────┼─────────┼────┼─────────■──┤ X ├┤ X ├──■──┤ X ├───────┼──»
       ├───┤  │    │         │    │         │  └─┬─┘├───┤  │  ├───┤┌───┐  │  »
 q1_2: ┤ H ├──┼────┼─────────■────┼─────────■────┼──┤ X ├──■──┤ X ├┤ X ├──┼──»
       └───┘┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │  └───┘┌─┴─┐└───┘└─┬─┘  │  »
 q1_3: ─────┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├───────■────┼──»
       ┌───┐└───┘┌───┐     └───┘┌───┐┌───┐└───┘          └───┘┌───┐       │  »
 q1_4: ┤ H ├──■──┤ X ├───────■──┤ X ├┤ X ├─────────────────■──┤ X ├───────■──»
       ├───┤  │  └─┬─┘       │  └─┬─┘└───┘     ┌───┐┌───┐  │  ├───┤       │  »
 q1_5: ┤ H ├──■────┼─────────┼────┼─────────■──┤ X ├┤ X ├──■──┤ X ├───────┼──»
       ├───┤  │    │         │    │  

In [6]:
# Execute the circuit on the qasm simulator
compiled_circuit = transpile(qc, simulator)
job = simulator.run(compiled_circuit, shots = 1000)
# job = execute(qc, simulator, shots=1000)

# Grab the results from the job
result = job.result()
print('end of execute')

counts = result.get_counts(compiled_circuit)
print(counts)
print('The solutions are giver by the strings that are not 0*')
if nColors>2:
    print('Please check: some of them may need LESS than', nColors, 'colours')
qc.draw()


end of execute
{'010100001': 10, '100001010': 22, '000000000': 884, '001010100': 24, '010001100': 19, '100010001': 24, '001100010': 17}
The solutions are giver by the strings that are not 0*
Please check: some of them may need LESS than 3 colours


┌───┐     ┌───┐          ┌───┐┌───┐                    ┌───┐          »
 q1_0: ┤ H ├──■──┤ X ├───────■──┤ X ├┤ X ├─────────────────■──┤ X ├───────■──»
       ├───┤  │  └─┬─┘       │  └─┬─┘└───┘     ┌───┐┌───┐  │  ├───┤       │  »
 q1_1: ┤ H ├──■────┼─────────┼────┼─────────■──┤ X ├┤ X ├──■──┤ X ├───────┼──»
       ├───┤  │    │         │    │         │  └─┬─┘├───┤  │  ├───┤┌───┐  │  »
 q1_2: ┤ H ├──┼────┼─────────■────┼─────────■────┼──┤ X ├──■──┤ X ├┤ X ├──┼──»
       └───┘┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │  └───┘┌─┴─┐└───┘└─┬─┘  │  »
 q1_3: ─────┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├───────■────┼──»
       ┌───┐└───┘┌───┐     └───┘┌───┐┌───┐└───┘          └───┘┌───┐       │  »
 q1_4: ┤ H ├──■──┤ X ├───────■──┤ X ├┤ X ├─────────────────■──┤ X ├───────■──»
       ├───┤  │  └─┬─┘       │  └─┬─┘└───┘     ┌───┐┌───┐  │  ├───┤       │  »
 q1_5: ┤ H ├──■────┼─────────┼────┼─────────■──┤ X ├┤ X ├──■──┤ X ├───────┼──»
       ├───┤  │    │         │    │         │  └─┬─┘├───┤  │  ├───┤┌───┐  │  »
 q1_6: ┤ H ├──┼────┼─────────■────┼─────────■────┼──┤ X ├──■──┤ X ├┤ X ├──┼──»
       └───┘┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │  └───┘┌─┴─┐└───┘└─┬─┘  │  »
 q1_7: ─────┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├───────■────┼──»
       ┌───┐└───┘┌───┐     └───┘┌───┐┌───┐└───┘          └───┘┌───┐       │  »
 q1_8: ┤ H ├──■──┤ X ├───────■──┤ X ├┤ X ├─────────────────■──┤ X ├───────┼──»
       ├───┤  │  └─┬─┘       │  └─┬─┘└───┘     ┌───┐┌───┐  │  ├───┤       │  »
 q1_9: ┤ H ├──■────┼─────────┼────┼─────────■──┤ X ├┤ X ├──■──┤ X ├───────┼──»
       ├───┤  │    │         │    │         │  └─┬─┘├───┤  │  ├───┤┌───┐  │  »
q1_10: ┤ H ├──┼────┼─────────■────┼─────────■────┼──┤ X ├──■──┤ X ├┤ X ├──┼──»
       └───┘┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │  └───┘┌─┴─┐└───┘└─┬─┘  │  »
q1_11: ─────┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├───────■────┼──»
            └───┘          └───┘          └───┘          └───┘          ┌─┴─┐»
q1_12: ─────────────────────────────────────────────────────────────────┤ X ├»
                                                                        └───┘»
q1_13: ──────────────────────────────────────────────────────────────────────»
                                                                             »
q1_14: ──────────────────────────────────────────────────────────────────────»
       ┌───┐                                                                 »
q1_15: ┤ X ├─────────────────────────────────────────────────────────────────»
       ├───┤                                                                 »
q1_16: ┤ X ├─────────────────────────────────────────────────────────────────»
       ├───┤                                                                 »
q1_17: ┤ X ├─────────────────────────────────────────────────────────────────»
       └───┘                                                                 »
 c0: 9/══════════════════════════════════════════════════════════════════════»
                                                                             »
«                                                         ┌───┐               »
« q1_0: ───────■──────────────────────────────────■───────┤ X ├───────────────»
«              │                                  │       └─┬─┘          ┌───┐»
« q1_1: ───────┼────■─────────■───────────────────┼─────────┼─────────■──┤ X ├»
«              │    │         │                   │         │         │  └─┬─┘»
« q1_2: ───────┼────┼─────────┼────■─────────■────┼─────────┼─────────┼────┼──»
«              │    │         │    │         │  ┌─┴─┐       │       ┌─┴─┐  │  »
« q1_3: ─|0>───┼────┼─────────┼────┼─────────┼──┤ X ├───────■───|0>─┤ X ├──■──»
«              │    │         │    │         │  └─┬─┘       │       └─┬─┘  │  »
« q1_4: ───────┼────┼────■────┼────┼─────────┼────┼─────────┼─────────┼────┼──»
«              │    │    │    │    │         │    │         │         │    │  »
« q1_5: ───────┼────■────┼────┼────┼────■────┼

In [7]:
sorted_items = sorted(counts.items(), key=lambda x: x[1], reverse=True)

print(sorted_items)

[('000000000', 884), ('001010100', 24), ('100010001', 24), ('100001010', 22), ('010001100', 19), ('001100010', 17), ('010100001', 10)]


In [8]:
import matplotlib.pyplot as plt
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeAuckland, FakeWashingtonV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [7]:
# If you did not previously save your credentials, use the following line instead:
# service = QiskitRuntimeService(channel="ibm_quantum", token="<MY_IBM_QUANTUM_TOKEN>")
backend = FakeAuckland()
# backend = service.least_busy(operational=True, simulator=False)
pass_manager = generate_preset_pass_manager(
    optimization_level=1,
    backend=backend,
    layout_method="trivial",  # Fixed layout mapped in circuit order
)

transpiled_circ = pass_manager.run(qc)
# sampler = Sampler(backend)
# job = sampler.run([transpiled_circ])
job = backend.run([compiled_circuit], shots = 1000)
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

# depths = []
# for _ in range(100):
#     depths.append(pass_manager.run(qc).depth())
 
# plt.figure(figsize=(8, 6))
# plt.hist(depths, align="left", color="#AC557C")
# plt.xlabel("Depth", fontsize=14)
# plt.ylabel("Counts", fontsize=14)

job id: 55f97d3b-4969-4456-92ce-0bd0d2de14c4
Result(backend_name='aer_simulator', backend_version='0.14.2', qobj_id='', job_id='55f97d3b-4969-4456-92ce-0bd0d2de14c4', success=True, results=[ExperimentResult(shots=1000, success=True, meas_level=2, data=ExperimentResultData(counts={'0x20': 10, '0x10a': 16, '0x8': 2, '0xa1': 19, '0x11': 3, '0x101': 1, '0x0': 847, '0x8c': 21, '0x54': 8, '0x111': 16, '0x40': 6, '0x62': 15, '0x110': 1, '0xcc': 2, '0x4': 3, '0x80': 5, '0x2': 6, '0x100': 5, '0x1': 6, '0x21': 1, '0x10': 3, '0xc0': 1, '0x74': 1, '0x84': 1, '0x102': 1}), header=QobjExperimentHeader(creg_sizes=[['c0', 9]], global_phase=0.0, memory_slots=9, n_qubits=18, name='circuit-166', qreg_sizes=[['q1', 18]], metadata={}), status=DONE, seed_simulator=436761263, metadata={'batched_shots_optimization': False, 'required_memory_mb': 4, 'method': 'statevector', 'active_input_qubits': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 'device': 'CPU', 'remapped_qubits': False, 'num_qubi

In [9]:
counts = result.get_counts(compiled_circuit)
print(counts)
sorted_items = sorted(counts.items(), key=lambda x: x[1], reverse=True)
print(sorted_items)

{'000100000': 10, '100001010': 16, '000001000': 2, '010100001': 19, '000010001': 3, '100000001': 1, '000000000': 847, '010001100': 21, '001010100': 8, '100010001': 16, '001000000': 6, '001100010': 15, '100010000': 1, '011001100': 2, '000000100': 3, '010000000': 5, '000000010': 6, '100000000': 5, '000000001': 6, '000100001': 1, '000010000': 3, '011000000': 1, '001110100': 1, '010000100': 1, '100000010': 1}
[('000000000', 847), ('010001100', 21), ('010100001', 19), ('100001010', 16), ('100010001', 16), ('001100010', 15), ('000100000', 10), ('001010100', 8), ('001000000', 6), ('000000010', 6), ('000000001', 6), ('010000000', 5), ('100000000', 5), ('000010001', 3), ('000000100', 3), ('000010000', 3), ('000001000', 2), ('011001100', 2), ('100000001', 1), ('100010000', 1), ('000100001', 1), ('011000000', 1), ('001110100', 1), ('010000100', 1), ('100000010', 1)]


In [9]:
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
pass_manager = generate_preset_pass_manager(
    optimization_level=1,
    backend=backend,
    layout_method="trivial",  # Fixed layout mapped in circuit order
)

transpiled_circ = pass_manager.run(qc)
# sampler = Sampler(backend)
# job = sampler.run([transpiled_circ])
job = backend.run([transpiled_circ], shots = 1000)
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

C:\Users\junji\AppData\Local\Temp\ipykernel_28452\3460214816.py:12: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  job = backend.run([transpiled_circ], shots = 1000)


job id: ctsk8ka3zkm0008s9edg


KeyboardInterrupt: 

In [ ]:
counts = result.get_counts(compiled_circuit)
print(counts)
sorted_items = sorted(counts.items(), key=lambda x: x[1], reverse=True)
print(sorted_items)